# VK-API: GEOtagged photos


by Bella Mironova based on the [materials]("https://gainanov.pro/rus-blog/programming/vk-favourite-places-for-photo/") of Ruslan Gaynanov


## 0. Import libraries


In [121]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import requests
from datetime import datetime
from time import sleep

## 1. Select GEOgraphical area and timeperiod


##### Define center coordinates, serach radius and timeperiod


In [72]:
coords = (60.7096, 28.7490)  # coordinates: latitude, longitude

dist = 3000  # search radius in meters

startDate = datetime(2023, 8, 1)  # first date to serach 
endDate = datetime(2023, 9, 1) # last date to serach 

timeperiod = (startDate.timestamp(), endDate.timestamp())

##### Define your vk token and vk version

Create you app and get a unique api key [here]("https://vk.com/editapp?act=create") ([instructions]("https://vk.com/dev/first_guide?f=2.%20Регистрация%20приложения"))


In [48]:
vk_access_token = 'YOUR_API_KEY' # your unique api key

vk_vers = 5.131 # current vk version

## 2. Search photos in a selected area and timeperiod


In [94]:

def getData(coords, timeperiod, offset):
    params = {
        'lat': coords[0],
        'long': coords[1],
        'count': '1000',
        'offset': offset,
        'radius': dist,
        'start_time': timeperiod[0],
        'end_time': timeperiod[1],
        'access_token': vk_access_token,
        'v': vk_vers,
        'sort': 0 # by date of creation
    }
    return requests.get("https://api.vk.com/method/photos.search",
                        params=params, verify=True).json()

## 3. Save response data


In [95]:
def savePoints(resp, df_points):
    try:
        items = resp['response']['items']
    except KeyError:
        return
    for f in items:
        try:
            df_points.loc[len(df_points.index)] = [f['id'], datetime.fromtimestamp(f['date']),f['text'],f['lat'],  f['long']]
        except KeyError:
            continue

## 4. Getting data and saving it to a DataFrame


In [96]:
df_points = pd.DataFrame(columns = ['id', 'date', 'text', 'lat', 'long'])
step = 24*60*60 # step = 1 day
i = timeperiod[0]

while i < timeperiod[1]:
    resp = getData(coords, (i, i+step), 0)
    savePoints(resp, df_points)
    count = resp['response']['count']
    returned = len(resp['response']['items'])
    if count > returned:
        offset = returned
        while offset < count and offset < 3000:
            resp = getData(coords, (i, i+step), offset)
            savePoints(resp, df_points)
            count = resp['response']['count']
            returned = len(resp['response']['items'])
            offset = offset + returned
            if returned == 0:
                break
    i = i + step
    sleep(0.5)      

## 5. Creating GeoDataFrame and visualizing result


##### Creating GeoDataFrame based on coordinates of points


In [113]:
map_data = gpd.GeoDataFrame(df_points, geometry=gpd.points_from_xy(df_points.long, df_points.lat), crs="EPSG:4326")

##### Creating heatmap to visualuze density of data points


In [125]:
osm_map = folium.Map(location=[map_data.lat.mean(), map_data.long.mean()], tiles="cartodb positron", zoom_start=14)
map_matrix = map_data[['lat', 'long']].values
HeatMap(map_matrix).add_to(osm_map)
osm_map

## 6. Saving gdb to file (optional)


In [119]:
map_data.to_file('vyborg_photos_0823.gpkg')